In [1]:
# First initialization

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import savemat
import os
import h5py as h5
import time
ax = {'x':1,'y':0,'z':2}
#define a dict of axes to make things simpler

In [2]:
from auspex.instruments import SR865
from pyanc350.v4 import Positioner

could not find anc350v2.dll
could not find anc350v3.dll


In [7]:
lockin = SR865()
lockin.connect('GPIB0::9::INSTR')   
#set time constant to 10 ms
anc = Positioner()
#instantiate positioner as anc
print('-------------------------------------------------------------')
print('capacitances:')
for axis in sorted(ax.keys()):
    print(axis,anc.measureCapacitance(ax[axis]))
    anc.setFrequency(ax[axis],200)
    anc.setAmplitude(ax[axis],15)

auspex-WARNING: 2022-05-23 11:11:20,816 ----> User-provided IP {} is a valid IP address but does not appear to be in human-readable format.
-------------------------------------------------------------
capacitances:
x 1.844093e-06
y 1.856622e-06
z 1.268291e-06


In [13]:
freq = 200
amp = 15
#assume that we are starting with the lens too far from the sample rather than too close
#we could also set this to be at x = 20mm to start to make sure sample
focus = 0
spot = 0
fine = 0
i = 0
data = [0] * 100000
anc.setAmplitude(ax['y'],amp)
anc.setFrequency(ax['y'],freq)
print('starting scan at',anc.getPosition(ax['y']))
anc.startContinuousMove(ax['y'],1, 1)
while focus == 0:
    i = i+1;
    time.sleep(0.01)
    data[i] = lockin.x
    if i > 10 and (data[i] - data[i-10])/data[i] > 0.05 and spot == 0:
        spot = 1
        print('signal increasing')
    if spot == 1 and (data[i] - data[i-2])/data[i] < -0.05:
        focus = 1
        anc.startContinuousMove(ax['y'],0,1)
        print('scan stopped at',anc.getPosition(ax['y']))

starting scan at 0.010159533
signal increasing
scan stopped at 0.008466174


In [5]:
# fine scan needs better calibration, doesn't really work right now, but will come back
fine = 0
anc.setFrequency(ax['x'],50)
anc.setAmplitude(ax['x'],25)
while fine == 0:
    i = i+1;
    print('stepping to maximize focus...')
    anc.startSingleStep(ax['x'],0)
    time.sleep(0.1)
    data[i] = lockin.x
    if data[i] < data[i-1]:
        fine = 1
        print('focused.')

NameError: name 'anc' is not defined

In [16]:
#cleanup

anc.disconnect()
lockin.disconnect()

KeyError: 2